In [1]:
import numpy as np
import torch
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import torch.nn as nn
from sklearn.metrics import matthews_corrcoef
import warnings
from sklearn import metrics
import torch.nn.init as init
warnings.filterwarnings("ignore")
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.metrics import precision_score,recall_score,accuracy_score
from sklearn.metrics import f1_score,matthews_corrcoef
from sklearn.feature_selection import VarianceThreshold
from scipy.stats import pearsonr
from sklearn.linear_model import ElasticNetCV
# from operator import itemgetter
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.utils import class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold,train_test_split,StratifiedKFold,KFold,cross_val_score,GroupKFold
from sklearn.feature_selection import RFE,VarianceThreshold,RFECV,SelectFromModel
from sklearn import metrics
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.ensemble import BaggingClassifier,StackingClassifier,VotingClassifier,GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import AdaBoostClassifier,StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
# from scipy.stats import pointbiserialr
from sklearn import preprocessing
from sklearn.base import clone
from sklearn.metrics import make_scorer,matthews_corrcoef
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV


In [2]:
per_index = pd.DataFrame(columns= ["Model","Dataset","Preprocessing","F1 (Toxic)","Precision (Toxic)","Sensitivity",
                                 "Specificity","BA","AUC-ROC","AUC-PR","MCC"])

per_index

,Model,Dataset,Preprocessing,F1 (Toxic),Precision (Toxic),Sensitivity,Specificity,BA,AUC-ROC,AUC-PR,MCC


In [3]:
def performance(per_index,model_name,dataset,preprocessing,y_true,y_pred):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    specificifty = tn/(tn+fp)
    BA = (recall + specificifty)/2
    x = (fp + tp)*(fp+tn)*(tp+fn)*(tn+fn)
    mcc = (tp*tn - fp*fn) / pow(x,0.5)
    f1 =  2 * (precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_true, y_pred)
    # 计算 AUC-PR
    pr_auc = average_precision_score(y_true, y_pred)
    data = {
    "Model": model_name,
    "Dataset": dataset,
    "Preprocessing": preprocessing,
    "F1 (Toxic)": f1,
    "Precision (Toxic)": precision,
    "Sensitivity": recall,
    "Specificity": specificifty,
    "BA": BA,
    "AUC-ROC": roc_auc,
    "AUC-PR": pr_auc,
    "MCC": mcc
    }
    per_index = pd.concat([per_index, pd.DataFrame([data])], ignore_index=True)
    return per_index
def MCC_Loss(y_pred, y_true, class_weights):
    tp = torch.sum((y_true * y_pred).float(), axis=0)
    tn = torch.sum(((1 - y_true) * (1 - y_pred)).float(), axis=0)
    fp = torch.sum(((1 - y_true) * y_pred).float(), axis=0)
    fn = torch.sum((y_true * (1 - y_pred)).float(), axis=0)
    x = (fp + tp) * (fp + tn) * (tp + fn) * (tn + fn) + 1e-7
    mcc = (tp * tn - fp * fn) / torch.pow(x, 0.5)

    mcc = torch.where(torch.isnan(mcc), torch.zeros_like(mcc), mcc)
    print(class_weights)
    weighted_mcc = mcc * class_weights  # 權重更新
    return 1 - torch.mean(weighted_mcc)

    
def f1_loss(y_pred,y_true):
    tp = torch.sum((y_true * y_pred).float(), axis=0)
    tn = torch.sum(((1 - y_true) * (1 - y_pred)).float(), axis=0)
    fp = torch.sum(((1 - y_true) * y_pred).float(), axis=0)
    fn = torch.sum((y_true * (1 - y_pred)).float(), axis=0)

    p = tp / (tp + fp + 1e-7)
    r = tp / (tp + fn + 1e-7)

    f1 = 2 * p * r / (p + r + 1e-7)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    return 1 - torch.mean(f1)
def recall_loss(y_pred,y_true):
    tp = torch.sum((y_true * y_pred).float(), axis=0)
    tn = torch.sum(((1 - y_true) * (1 - y_pred)).float(), axis=0)
    fp = torch.sum(((1 - y_true) * y_pred).float(), axis=0)
    fn = torch.sum((y_true * (1 - y_pred)).float(), axis=0)

    p = tp / (tp + fp + 1e-7)
    r = tp / (tp + fn + 1e-7)

    f1 = 2 * p * r / (p + r + 1e-7)
    r = torch.where(torch.isnan(r), torch.zeros_like(r), r)
    return 1 - torch.mean(r)
def sp_loss(y_pred,y_true):
    tp = torch.sum((y_true * y_pred).float(), axis=0)
    tn = torch.sum(((1 - y_true) * (1 - y_pred)).float(), axis=0)
    fp = torch.sum(((1 - y_true) * y_pred).float(), axis=0)
    fn = torch.sum((y_true * (1 - y_pred)).float(), axis=0)

    sp = tn / (tn + fp + 1e-7)


    sp = torch.where(torch.isnan(sp), torch.zeros_like(sp), sp)
    return 1 - torch.mean(sp)


In [4]:
def calculate_class_weights(y_train):
    unique, counts = np.unique(y_train, return_counts=True)
    total_samples = len(y_train)
    class_weights = total_samples / counts
    return torch.tensor(class_weights, dtype=torch.float32)
seed = 42
torch.manual_seed(seed)
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
#         self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = torch.nn.Linear(hidden_size, 64)
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.4)
        self.fc2 = torch.nn.Linear(64, 8)
        self.bn2 = torch.nn.BatchNorm1d(8)
        self.fc3 = torch.nn.Linear(8,1)

     
   


        # 使用 He 初始化
        for name, param in self.rnn.named_parameters():
            if 'weight' in name:
                init.kaiming_normal_(param, mode='fan_in', nonlinearity='relu')
        init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        init.kaiming_normal_(self.fc3.weight, mode='fan_in', nonlinearity='relu')

    def forward(self, x):
#         x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc1(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.dropout(out)
#         out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc3(out)
#         out = self.dropout2(out)
        out = self.relu(out)
        out = torch.sigmoid(out)
        return out.squeeze(1)

In [5]:
def train(model, optimizer, train_loader):
    model.train()
    running_loss = 0.0
    correct = 0.0
    total = 0
    y_true = []
    y_pred = []
    for i, (inputs, labels) in enumerate(train_loader):

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = MCC_Loss(outputs, labels,calculate_class_weights(y_train))
        l1_lambda = 0.000045
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = loss + l1_lambda * l1_norm

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = torch.round(outputs).squeeze().cpu().detach().numpy()
        predicted = predicted.flatten()
        total += labels.size(0)
        labels = labels.cpu().numpy()
        correct += (predicted == labels).sum().item()
        y_true.extend(labels)
        y_pred.extend(predicted.tolist())
    train_loss = running_loss / len(train_loader)
    t_f1 = f1_score(y_true, y_pred)
    t_mcc = matthews_corrcoef(y_true, y_pred)
    tcombined_score = (t_f1 + t_mcc) / 2
    mtr = metrics.confusion_matrix(y_true, y_pred)
    return train_loss,t_f1,t_mcc,tcombined_score,mtr

def validate(model, val_loader, l2_lambda):
    model.eval()
    running_loss = 0.0
    correct = 0.0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = MCC_Loss(outputs, labels,calculate_class_weights(y_train))
            l1_lambda = 0.000045
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss = loss + l1_lambda * l1_norm

            running_loss += loss.item()
            predicted = torch.round(outputs).squeeze().cpu().detach().numpy()
            predicted = predicted.flatten()
            total += labels.size(0)
            labels = labels.cpu().numpy()
            correct += (predicted == labels).sum().item()
            y_true.extend(labels)
            y_pred.extend(predicted.tolist())
        val_loss = running_loss / len(val_loader)
        val_f1 = f1_score(y_true, y_pred)
        val_mcc = matthews_corrcoef(y_true, y_pred)
        mtr = metrics.confusion_matrix(y_true, y_pred)
        combined_score = (val_f1 + val_mcc) / 2
    return val_loss, val_f1, val_mcc, combined_score,mtr
def test(model, test_loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss =  MCC_Loss(outputs, labels,calculate_class_weights(y_test)) 
            l1_lambda = 0.000045
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss = loss + l1_lambda * l1_norm
            running_loss += loss.item()
            predicted = torch.round(outputs).squeeze().cpu().detach().numpy()
            predicted = predicted.flatten()
            total += labels.size(0)
            labels = labels.cpu().numpy()
            correct += (predicted == labels).sum().item()
            y_true.extend(labels)
            y_pred.extend(predicted.tolist())
    test_loss = running_loss / len(test_loader)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    mtr = metrics.confusion_matrix(y_true, y_pred)
    return test_loss, f1,mcc,mtr,y_true,y_pred

In [39]:
def PCA_func(fingerprint1,fingerprint2):

    print(np.array(fingerprint1).shape)
    print(np.array(fingerprint2).shape)
    # 將PCA結果轉換為DataFrame
    train_pca_df = pd.DataFrame(data=fingerprint1, columns=['PC1', 'PC2'])
    test_pca_df = pd.DataFrame(data=fingerprint2, columns=['PC1', 'PC2'])

    # 繪製化學空間分佈圖
    plt.scatter(train_pca_df['PC1'], train_pca_df['PC2'], c='blue', label='Non-Toxicity')
    plt.scatter(test_pca_df['PC1'], test_pca_df['PC2'], c='red', label='Toxicity')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Chemical Space Distribution (Unbalanced)')
    plt.legend()
    plt.show()

In [21]:
morgan_data = pd.read_csv("data/morgan_data.csv",index_col=0)
morgan_test = pd.read_csv("data/morgan_test.csv",index_col=0)
mordred_data= pd.read_csv("data/mordred_data.csv",index_col=0)
mordred_test= pd.read_csv("data/mordred_test.csv",index_col=0)

In [40]:
#check if it is a value
for col in mordred_data.columns:
    if not pd.api.types.is_numeric_dtype(mordred_data[col]):
        mordred_data[col] = pd.to_numeric(mordred_data[col], errors='coerce')
        mordred_test[col] = pd.to_numeric(mordred_test[col], errors='coerce')
# for col in mordred_test.columns:
#     if not pd.api.types.is_numeric_dtype(mordred_test[col]):
#         mordred_test[col] = pd.to_numeric(mordred_test[col], errors='coerce')



In [41]:
#fill the nan value

# mordred_data = mordred_data.dropna(axis=1, how='all')
# mordred_test = mordred_test.dropna(axis=1, how='all')

for col in mordred_data.columns:
    if mordred_data[col].isna().sum() >= 1:
        mordred_data[col].fillna(mordred_data[col].mean(), inplace=True)
        mordred_test[col].fillna(mordred_test[col].mean(), inplace=True)
# for col in mordred_test.columns:
#     if mordred_test[col].isna().sum() >= 1:
#         mordred_test[col].fillna(mordred_test[col].mean(), inplace=True)

In [42]:
#fill the nan value

mordred_data = mordred_data.dropna(axis=1, how='all')
mordred_test = mordred_test.dropna(axis=1, how='all')





In [43]:
# 找出mordred_data和mordred_test之间差异的列
different_columns = set(mordred_data.columns) - set(mordred_test.columns)

# 删除mordred_data中的差异列
mordred_data = mordred_data.drop(columns=different_columns)

# 找出mordred_test和mordred_data之间差异的列（因为在上一步中mordred_data的列已经改变了）
different_columns = set(mordred_test.columns) - set(mordred_data.columns)

# 删除mordred_test中的差异列
mordred_test = mordred_test.drop(columns=different_columns)


In [48]:
morgan_X_train,morgan_X_test,morgan_y_train,morgan_y_test = train_test_split(np.array(morgan_data.iloc[:,:-1]),np.array(morgan_data.iloc[:,-1]),stratify=np.array(morgan_data.iloc[:,-1]),test_size=0.2)

In [50]:
pd.Series(morgan_y_train).value_counts()

0    4774
1    1234
Name: count, dtype: int64

In [52]:
pca = PCA()

# PCA
pca.fit(morgan_X_train)

# 計算PCA各個主成份的方差解釋率
explained_variance_ratio = pca.explained_variance_ratio_

# 累積方差解釋率
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# setting the threshold
threshold = 0.95

# 找到最小主成分数目
n_components = np.where(cumulative_explained_variance_ratio >= threshold)[0][0] + 1

print(f"Number of components to preserve {threshold * 100}% variance: {n_components}")

pca_reduced = PCA(n_components=n_components)
# 進行 PCA 轉換
X_train = pca_reduced.fit_transform(morgan_X_train)
# X_val1 = pca_reduced.transform(X_val)
X_test = pca_reduced.transform(morgan_X_test)

positive_indices = np.where(morgan_y_train == 1)[0]
negative_indices = np.where(morgan_y_train == 0)[0]

positive_samples = np.array(X_train)[positive_indices]
negative_samples = np.array(X_train)[negative_indices]
overlaps = []
for i in range(positive_samples.shape[1]):  # for each feature
    hist_pos, _ = np.histogram(positive_samples[:, i])
    hist_neg, _ = np.histogram(negative_samples[:, i])
    overlap = np.minimum(hist_pos, hist_neg).sum()  # calculate the overlap for this feature
    overlaps.append(overlap)

overlap_ratio = np.mean(overlaps) / positive_samples.shape[0]  # calculate the average overlap
print("Average overlap ratio:", overlap_ratio)

Number of components to preserve 95.0% variance: 1106
Average overlap ratio: 0.9406193123701279


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
clf = RandomForestClassifier(random_state=0,class_weight={0: 1234/4774, 1: 1})

# 計算每個特徵與目標變數的相關性
F, p = f_regression(morgan_X_train, morgan_y_train)

# 決定我們要試驗的特徵數量
k_values = range(1, morgan_X_train.shape[1] + 1)

best_score = 0
best_k = 0
best_X = None

# 在迴圈開始前初始化兩個空列表
n_list = []
scores_list = []

for k in k_values:
    # 選擇 k 個最相關的特徵
    selection = SelectKBest(f_regression, k=k)
    select_X = selection.fit_transform(morgan_X_train, morgan_y_train)
    
    # 進行交叉驗證
    scores = cross_val_score(clf, select_X, morgan_y_train, cv=5, scoring='f1')
    mean_score = scores.mean()
    
    print(f"k={k}, n={select_X.shape[1]}, Accuracy: {mean_score:.2f} (+/- {scores.std():.2f})")

    # 如果這個特徵數量的得分比我們之前找到的得分還高，則更新最佳特徵數量和數據集
    if mean_score > best_score:
        best_score = mean_score
        best_k = k
        best_X = select_X

    # 將每個特徵數量和對應的評分添加到列表中
    n_list.append(select_X.shape[1])
    scores_list.append(mean_score)

print(f"Best k={best_k}, Accuracy: {best_score:.2f}")


k=1, n=1, Accuracy: 0.36 (+/- 0.02)
k=2, n=2, Accuracy: 0.43 (+/- 0.02)
k=3, n=3, Accuracy: 0.44 (+/- 0.02)
k=4, n=4, Accuracy: 0.43 (+/- 0.02)
k=5, n=5, Accuracy: 0.44 (+/- 0.02)
k=6, n=6, Accuracy: 0.43 (+/- 0.02)
k=7, n=7, Accuracy: 0.43 (+/- 0.02)
k=8, n=8, Accuracy: 0.43 (+/- 0.02)
k=9, n=9, Accuracy: 0.44 (+/- 0.02)
k=10, n=10, Accuracy: 0.43 (+/- 0.02)
k=11, n=11, Accuracy: 0.44 (+/- 0.01)
k=12, n=12, Accuracy: 0.44 (+/- 0.01)
k=13, n=13, Accuracy: 0.46 (+/- 0.02)
k=14, n=14, Accuracy: 0.46 (+/- 0.02)
k=15, n=15, Accuracy: 0.46 (+/- 0.02)
k=16, n=16, Accuracy: 0.46 (+/- 0.01)
k=17, n=17, Accuracy: 0.46 (+/- 0.01)
k=18, n=18, Accuracy: 0.46 (+/- 0.02)
k=19, n=19, Accuracy: 0.47 (+/- 0.02)
k=20, n=20, Accuracy: 0.47 (+/- 0.01)
k=21, n=21, Accuracy: 0.46 (+/- 0.01)
k=22, n=22, Accuracy: 0.47 (+/- 0.01)
k=23, n=23, Accuracy: 0.47 (+/- 0.01)
k=24, n=24, Accuracy: 0.47 (+/- 0.02)
k=25, n=25, Accuracy: 0.47 (+/- 0.01)
k=26, n=26, Accuracy: 0.47 (+/- 0.01)
k=27, n=27, Accuracy: 0.47 (+/

k=212, n=212, Accuracy: 0.55 (+/- 0.02)
k=213, n=213, Accuracy: 0.54 (+/- 0.03)
k=214, n=214, Accuracy: 0.54 (+/- 0.03)
k=215, n=215, Accuracy: 0.54 (+/- 0.03)
k=216, n=216, Accuracy: 0.54 (+/- 0.02)
k=217, n=217, Accuracy: 0.54 (+/- 0.03)
k=218, n=218, Accuracy: 0.54 (+/- 0.03)
k=219, n=219, Accuracy: 0.54 (+/- 0.03)
k=220, n=220, Accuracy: 0.55 (+/- 0.03)
k=221, n=221, Accuracy: 0.55 (+/- 0.03)
k=222, n=222, Accuracy: 0.54 (+/- 0.03)
k=223, n=223, Accuracy: 0.54 (+/- 0.02)
k=224, n=224, Accuracy: 0.55 (+/- 0.02)
k=225, n=225, Accuracy: 0.54 (+/- 0.02)
k=226, n=226, Accuracy: 0.55 (+/- 0.03)
k=227, n=227, Accuracy: 0.54 (+/- 0.03)
k=228, n=228, Accuracy: 0.55 (+/- 0.02)
k=229, n=229, Accuracy: 0.54 (+/- 0.03)
k=230, n=230, Accuracy: 0.54 (+/- 0.03)
k=231, n=231, Accuracy: 0.54 (+/- 0.04)
k=232, n=232, Accuracy: 0.55 (+/- 0.02)
k=233, n=233, Accuracy: 0.53 (+/- 0.02)
k=234, n=234, Accuracy: 0.55 (+/- 0.02)
k=235, n=235, Accuracy: 0.54 (+/- 0.03)
k=236, n=236, Accuracy: 0.54 (+/- 0.02)


k=417, n=417, Accuracy: 0.55 (+/- 0.03)
k=418, n=418, Accuracy: 0.55 (+/- 0.03)
k=419, n=419, Accuracy: 0.55 (+/- 0.03)
k=420, n=420, Accuracy: 0.54 (+/- 0.01)
k=421, n=421, Accuracy: 0.54 (+/- 0.03)
k=422, n=422, Accuracy: 0.54 (+/- 0.03)
k=423, n=423, Accuracy: 0.54 (+/- 0.03)
k=424, n=424, Accuracy: 0.54 (+/- 0.03)
k=425, n=425, Accuracy: 0.54 (+/- 0.02)
k=426, n=426, Accuracy: 0.54 (+/- 0.03)
k=427, n=427, Accuracy: 0.54 (+/- 0.02)
k=428, n=428, Accuracy: 0.54 (+/- 0.02)
k=429, n=429, Accuracy: 0.53 (+/- 0.03)
k=430, n=430, Accuracy: 0.54 (+/- 0.02)
k=431, n=431, Accuracy: 0.55 (+/- 0.02)
k=432, n=432, Accuracy: 0.54 (+/- 0.03)
k=433, n=433, Accuracy: 0.54 (+/- 0.02)
k=434, n=434, Accuracy: 0.55 (+/- 0.03)
k=435, n=435, Accuracy: 0.54 (+/- 0.03)
k=436, n=436, Accuracy: 0.54 (+/- 0.02)
k=437, n=437, Accuracy: 0.53 (+/- 0.04)
k=438, n=438, Accuracy: 0.53 (+/- 0.03)
k=439, n=439, Accuracy: 0.54 (+/- 0.03)
k=440, n=440, Accuracy: 0.54 (+/- 0.03)
k=441, n=441, Accuracy: 0.54 (+/- 0.03)


In [ ]:
print(f"Best threshold={best_threshold}, Accuracy: {best_score:.2f}")

# 使用 matplotlib 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(n_list, scores_list, marker='o')
plt.title('Score vs. Feature Importance Threshold')
plt.xlabel('Feature Importance Threshold')
plt.ylabel('Cross-Validated Score')
plt.grid()
plt.show()

In [ ]:
selection = SelectKBest(score_func=mutual_info_classif, k= best_k)
X_train_k = selection.fit_transform(morgan_X_train,morgan_y_train)
X_test_k = selection.transform(morgan_X_test)
print(np.array(X_train_k).shape,np.array(X_test_k).shape)

In [ ]:
pca = PCA()

# PCA
pca.fit(X_train_k)

# 計算PCA各個主成份的方差解釋率
explained_variance_ratio = pca.explained_variance_ratio_

# 累積方差解釋率
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# setting the threshold
threshold = 0.95

# 找到最小主成分数目
n_components = np.where(cumulative_explained_variance_ratio >= threshold)[0][0] + 1

print(f"Number of components to preserve {threshold * 100}% variance: {n_components}")

pca_reduced = PCA(n_components=n_components)
# 進行 PCA 轉換
X_train = pca_reduced.fit_transform(X_train_k)
# X_val1 = pca_reduced.transform(X_val)
X_test = pca_reduced.transform(X_test_k)

positive_indices = np.where(y_train == 1)[0]
negative_indices = np.where(y_train == 0)[0]

positive_samples = np.array(X_train)[positive_indices]
negative_samples = np.array(X_train)[negative_indices]
overlaps = []
for i in range(positive_samples.shape[1]):  # for each feature
    hist_pos, _ = np.histogram(positive_samples[:, i])
    hist_neg, _ = np.histogram(negative_samples[:, i])
    overlap = np.minimum(hist_pos, hist_neg).sum()  # calculate the overlap for this feature
    overlaps.append(overlap)

overlap_ratio = np.mean(overlaps) / positive_samples.shape[0]  # calculate the average overlap
print("Average overlap ratio:", overlap_ratio)


In [ ]:




# 初始化 k 折交叉驗證
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

X = X_train_k
y = morgan_y_train

# 訓練和評估
result_f1 = []
result_mcc = []
total_y_true = []
total_y_pred = []

f1_four = []
mcc_four = []
total_y_true_four = []
total_y_pred_four = []
for fold, (val_indices, test_indices) in enumerate(kfold.split(X, y)):
    # 分割數據集
    X_train, X_val = X[val_indices], X[test_indices]
    y_train, y_val = y[val_indices], y[test_indices]
    
    # 將訓練集再分為訓練集和驗證集
    #X_train1, X_val, y_train1, y_val = train_test_split(X_train1, y_train1, test_size=0.25, random_state=42, stratify=y_train1)
    X_train = csr_matrix(X_train).toarray()


    
#     negative_indices = np.where(labels == 0)[0]
#     negative_samples = np.array(np.array(feature))[negative_indices]
#     negative_labels  = np.array(np.array(labels))[negative_indices]
#     print(pd.Series(negative_labels).value_counts())


    # 假设 X 是您的数据矩阵，形状为 (样本数, 特征数)，例如 (500, 2048)

    # 初始化 PCA，所有主成分
    pca = PCA()

    # PCA
    pca.fit(X_train)

    # 計算PCA各個主成份的方差解釋率
    explained_variance_ratio = pca.explained_variance_ratio_

    # 累積方差解釋率
    cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

    # setting the threshold
    threshold = 0.95

    # 找到最小主成分数目
    n_components = np.where(cumulative_explained_variance_ratio >= threshold)[0][0] + 1

    print(f"Number of components to preserve {threshold * 100}% variance: {n_components}")
    
    pca_reduced = PCA(n_components=n_components)
    # 進行 PCA 轉換
    X_train = pca_reduced.fit_transform(X_train)
    X_val = pca_reduced.transform(X_val)
    X_test = pca_reduced.transform(morgan_X_test)
#     X_four = pca_reduced.transform(np.array(feature))

    # 轉換標籤為一維張量
    y_train = y_train.flatten()
    y_val1 = y_val.flatten()
    y_test = morgan_y_train.flatten()
#     y_four = labels.flatten()

    # 創建數據加載器
    train_data = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
    val_data = torch.utils.data.TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
    test_data = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
#     four_data = torch.utils.data.TensorDataset(torch.Tensor(X_four), torch.Tensor(y_four))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size =32, shuffle=True, drop_last=True)
#     four_loader = torch.utils.data.DataLoader(four_data, batch_size =32, shuffle=True, drop_last=True)

    # 實例化模型、損失函數和優化器

    input_size = 2048
    embedding_dim = n_components
    hidden_size = 128
    output_size = 1 #二元分類問題
    num_layers = 4
    model = RNNModel(input_size,embedding_dim, hidden_size, output_size, num_layers).to(dtype=torch.float)

    learning_rate = 0.0005

    l2_lambda = 0.00005
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)

    patience = 30
    counter = 0
    best_val_loss = float('inf')


    n_epochs = 1000


    # Initialize lists to store training and validation loss and F1 score
    train_losses = []
    val_losses = []
    train_f1_scores = []
    train_mcc_scores = []
    val_f1_scores = []
    val_mcc_scores = []

    for epoch in range(n_epochs):
        train_loss,t_f1,t_mcc,tcombined_score,train_mtr =  train(model, optimizer, train_loader)
        val_loss, val_f1, val_mcc, combined_score,mtr = validate(model, val_loader, l2_lambda)
        scheduler.step(val_loss)
        # Append the results to the lists
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_f1_scores.append(t_f1)
        train_mcc_scores.append(t_mcc)
        val_f1_scores.append(val_f1)
        val_mcc_scores.append(val_mcc)   
        print('Epoch [{}/{}], Training Loss: {:.4f}, Training F1 score: {:.4f}, Training MCC score: {:.4f}, Training Combine score: {:.4f}'.format(epoch+1, n_epochs, train_loss,t_f1,t_mcc,tcombined_score))
        print(train_mtr)
        print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation F1 score: {:.4f}, Validation MCC score: {:.4f}, Validation Combine score: {:.4f}'.format(epoch+1, n_epochs,val_loss, val_f1, val_mcc, combined_score))
        print(mtr)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered at epoch {}.".format(epoch+1))
                break





    test_loss, f1,mcc,mtr,y_true,y_pred = test(model, test_loader)
    print('Test Loss: {:.4f}, F1 Score: {:.2f}, MCC: {:.2f}, Combine: {:.2f}'.format(test_loss, f1,mcc,(f1+mcc)/2))
    print(mtr)
    
    total_y_pred = total_y_pred + y_pred
    total_y_true = total_y_true + y_true
    
#     test_loss, f1,mcc,mtr,y_true,y_pred = test(model, four_loader)
#     print('Test Loss: {:.4f}, Accuracy Score: {:.2f}'.format(test_loss,accuracy_score(y_true, y_pred)))
#     print(mtr)

#     total_y_pred_four = total_y_pred_four + y_pred
#     total_y_true_four = total_y_true_four + y_true
    
    # Plotting the training and validation loss
    plt.figure()
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

    # Plotting the training F1 score
    plt.figure()
    plt.plot(train_f1_scores, label='F1 Score', linestyle='-', marker='o')
    plt.plot(train_mcc_scores, label='MCC Score', linestyle='--', marker='x')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.title('Training F1 and MCC Scores')
    plt.legend()
    plt.show()

    # Plotting the validation F1 score
    plt.figure()
    plt.plot(val_f1_scores, label='F1 Score', linestyle='-', marker='o')
    plt.plot(val_mcc_scores, label='MCC Score', linestyle='--', marker='x')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.title('Validation F1 and MCC Scores')
    plt.legend()
    plt.show()


model_name = "RNN"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,total_y_true,total_y_pred)
# print("Testing for all negative : {}".format(accuracy_score(total_y_true_four, total_y_pred_four)))
per_index
# model_name = "RNN"
# dataset = "MorganFingerPrint (ALL data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,total_y_true_four,total_y_pred_four)

In [ ]:
# 初始化 PCA，所有主成分
pca = PCA()

# PCA
pca.fit(X_train_k)

# 計算PCA各個主成份的方差解釋率
explained_variance_ratio = pca.explained_variance_ratio_

# 累積方差解釋率
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# setting the threshold
threshold = 0.95

# 找到最小主成分数目
n_components = np.where(cumulative_explained_variance_ratio >= threshold)[0][0] + 1

print(f"Number of components to preserve {threshold * 100}% variance: {n_components}")

pca_reduced = PCA(n_components=n_components)
# 進行 PCA 轉換
X_train = pca_reduced.fit_transform(X_train_k)
# X_val1 = pca_reduced.transform(X_val)
X_test = pca_reduced.transform(X_test_k)

# 轉換標籤為一維張量
y_train = morgan_y_train.ravel()
y_test = np.array(morgan_y_test).ravel()
# y_four = np.array(labels).ravel()
print("Size of training set : {}   {}".format(X_train.shape,y_train.shape))



def mcc_score(y_true, y_pred):
    return (matthews_corrcoef(y_true, y_pred)+f1_score(y_true, y_pred))/2

# 将 MCC 转换为评估指标
scoring = make_scorer(mcc_score)

X_train_copy = X_train
X_test_copy = X_test


k_best_selector = SelectKBest(k=100)
percentile_selector = SelectPercentile(percentile=100)
pca_selector = PCA(n_components=100)
selector = VarianceThreshold(threshold=0)
svm = SVC(kernel='linear', C=1, random_state=42)
selector = SelectFromModel(svm, threshold=0.3)
rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold= "mean")
n_iter_search = 20 



nb_classifier = GaussianNB()
params_nb = {'classifier__var_smoothing': np.logspace(0,-9, num=100)}
pipeline_nb = Pipeline([
    ('classifier', nb_classifier)
])
model_nb = RandomizedSearchCV(pipeline_nb, param_distributions=params_nb,
                              n_iter=n_iter_search, verbose=1,
                              scoring=scoring, cv=5)
model_nb.fit(X_train_copy, y_train)
nb = model_nb.best_estimator_
matrix = metrics.confusion_matrix(y_test,nb.predict(X_test_copy))
precision = precision_score(y_test,nb.predict(X_test_copy))
recall = recall_score(y_test,nb.predict(X_test_copy))
f1 = f1_score(y_test,nb.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,nb.predict(X_test_copy))

model_name = "GaussianNB"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,nb.predict(X_test_copy))

print("########################NB########################")
print(" Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)

# matrix = metrics.confusion_matrix(y_four,nb.predict(X_four))
# precision = precision_score(y_four,nb.predict(X_four))
# recall = recall_score(y_four,nb.predict(X_four))
# f1 = f1_score(y_four,nb.predict(X_four))
# mcc = matthews_corrcoef(y_four,nb.predict(X_four))

# model_name = "GaussianNB"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,nb.predict(X_four))

# print("########################NB########################")
# print(" Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test

params_log = {'classifier__penalty':['l2','l1'],"classifier__C" :[0.0001,0.001, 0.01, 0.1, 1, 10, 100],
    "classifier__solver":["liblinear"],"classifier__max_iter":[50,100,200,500,700,1000],
             "classifier__class_weight" : [{0: 1234/4774, 1: 1}]}
log_classifier = LogisticRegression()
pipeline_log = Pipeline([
    ('classifier', log_classifier)
])
model_log = RandomizedSearchCV(pipeline_log,
                               param_distributions=params_log,
                               n_iter=n_iter_search, verbose=1,
                               scoring=scoring, cv=5)
model_log.fit(X_train_copy,y_train)
logg = model_log.best_estimator_
matrix = metrics.confusion_matrix(y_test,logg.predict(X_test_copy))
precision = precision_score(y_test,logg.predict(X_test_copy))
recall = recall_score(y_test,logg.predict(X_test_copy))
f1 = f1_score(y_test,logg.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,logg.predict(X_test_copy))

model_name = "LogisticRegression"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,logg.predict(X_test_copy))

print("########################LogisticRegression########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)

# matrix = metrics.confusion_matrix(y_four,logg.predict(X_four))
# precision = precision_score(y_four,logg.predict(X_four))
# recall = recall_score(y_four,logg.predict(X_four))
# f1 = f1_score(y_four,logg.predict(X_four))
# mcc = matthews_corrcoef(y_four,logg.predict(X_four))

# model_name = "LogisticRegression"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,logg.predict(X_four))

# print("########################LogisticRegression########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test


params_svm = {'classifier__kernel':['linear', 'poly', 'rbf','sigmoid'],
              "classifier__C" :[0.0001,0.001, 0.01, 0.1, 1, 10, 100],
            "classifier__gamma" : ['scale','auto'],
             "classifier__class_weight" : [{0: 1234/4774, 1: 1}]}
svm_classifier = SVC()
pipeline_svm = Pipeline([
    ('classifier', svm_classifier)
])
model_svm = RandomizedSearchCV(pipeline_svm,
                               param_distributions=params_svm,
                               n_iter=n_iter_search, verbose=1,
                               scoring=scoring, cv=5)
model_svm.fit(X_train_copy,y_train)
svm = model_svm.best_estimator_
matrix = metrics.confusion_matrix(y_test,svm.predict(X_test_copy))
precision = precision_score(y_test,svm.predict(X_test_copy))
recall = recall_score(y_test,svm.predict(X_test_copy))
f1 = f1_score(y_test,svm.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,svm.predict(X_test_copy))

model_name = "SVM"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,svm.predict(X_test_copy))

print("########################SVM########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,svm.predict(X_four))
# precision = precision_score(y_four,svm.predict(X_four))
# recall = recall_score(y_four,svm.predict(X_four))
# f1 = f1_score(y_four,svm.predict(X_four))
# mcc = matthews_corrcoef(y_four,svm.predict(X_four))

# model_name = "SVM"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,svm.predict(X_four))

# print("########################SVM########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)

X_train_copy = X_train
X_test_copy = X_test


params_xgb = {'classifier__learning_rate':[0.01,0.001,0.0001],
              'classifier__max_depth': range(10,100,20),
              "classifier__n_estimators" :range(1,201,50),
              "classifier__gamma" :range(1,10,3),
              "classifier__scale_pos_weight":[(4774/1234)]
             }
xgb_classifier = XGBClassifier()
pipeline_xgb = Pipeline([
    ('classifier', xgb_classifier)
])
model_xgb = RandomizedSearchCV(pipeline_xgb,
                               param_distributions=params_xgb,
                               n_iter=n_iter_search, verbose=1,
                               scoring=scoring, cv=5)
model_xgb.fit(X_train_copy,y_train)
xgb = model_xgb.best_estimator_
matrix = metrics.confusion_matrix(y_test,xgb.predict(X_test_copy))
precision = precision_score(y_test,xgb.predict(X_test_copy),zero_division=0)
recall = recall_score(y_test,xgb.predict(X_test_copy))
f1 = f1_score(y_test,xgb.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,xgb.predict(X_test_copy))

model_name = "XGBClassifier"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,xgb.predict(X_test_copy))


print("########################XGBClassifier########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,xgb.predict(X_four))
# precision = precision_score(y_four,xgb.predict(X_four),zero_division=0)
# recall = recall_score(y_four,xgb.predict(X_four))
# f1 = f1_score(y_four,xgb.predict(X_four))
# mcc = matthews_corrcoef(y_four,xgb.predict(X_four))

# model_name = "XGBClassifier"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,xgb.predict(X_four))


# print("########################XGBClassifier########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test


params_rf = {'classifier__max_features':['sqrt','log2'],
              'classifier__max_depth': range(10,100,20),
              "classifier__n_estimators" :range(100,1000,100),
   "classifier__criterion" : ['gini','entropy'],
             "classifier__oob_score" : [True],
             "classifier__class_weight" : [{0: 1234/4774, 1: 1}]
             
             }
rf_classifier = RandomForestClassifier()
pipeline_rf = Pipeline([
    ('classifier', rf_classifier)
])
model_rf = RandomizedSearchCV(pipeline_rf,
                              param_distributions=params_rf,
                              n_iter=n_iter_search, verbose=1,
                              scoring=scoring, cv=5)
model_rf.fit(X_train_copy,y_train)
rf = model_rf.best_estimator_
matrix = metrics.confusion_matrix(y_test,rf.predict(X_test_copy))
precision = precision_score(y_test,rf.predict(X_test_copy),zero_division=1)
recall = recall_score(y_test,rf.predict(X_test_copy))
f1 = f1_score(y_test,rf.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,rf.predict(X_test_copy))

model_name = "RandomForestClassifier"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,rf.predict(X_test_copy))

print("########################RandomForest########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,rf.predict(X_four))
# precision = precision_score(y_four,rf.predict(X_four),zero_division=1)
# recall = recall_score(y_four,rf.predict(X_four))
# f1 = f1_score(y_four,rf.predict(X_four))
# mcc = matthews_corrcoef(y_four,rf.predict(X_four))

# model_name = "RandomForestClassifier"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,rf.predict(X_four))

# print("########################RandomForest########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test



params_lgb = {'classifier__objective': ["binary"],
              "classifier__learning_rate" :[0.01, 0.015, 0.025, 0.05, 0.1],
   "classifier__max_depth" : [3, 5, 6, 7, 9, 12, 15, 17, 25],
             "classifier__subsample" : [0.6, 0.7, 0.8, 0.9, 1],
              "classifier__class_weight" : [{0: 1234/4774, 1: 1}]
              
             }
lgb_classifier = lgb.LGBMClassifier()
pipeline_lgb = Pipeline([
    ('classifier', lgb_classifier)
])
model_lgb = RandomizedSearchCV(pipeline_lgb,
                               param_distributions=params_lgb,
                               n_iter=n_iter_search, verbose=1,
                               scoring=scoring, cv=5)

model_lgb.fit(X_train_copy,y_train)
lgb = model_lgb.best_estimator_
matrix = metrics.confusion_matrix(y_test,lgb.predict(X_test_copy))
precision = precision_score(y_test,lgb.predict(X_test_copy))
recall = recall_score(y_test,lgb.predict(X_test_copy))
f1 = f1_score(y_test,lgb.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,lgb.predict(X_test_copy))

model_name = "LGBMClassifier"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,lgb.predict(X_test_copy))

print("########################LGBMClassifier########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,lgb.predict(X_four))
# precision = precision_score(y_four,lgb.predict(X_four))
# recall = recall_score(y_four,lgb.predict(X_four))
# f1 = f1_score(y_four,lgb.predict(X_four))
# mcc = matthews_corrcoef(y_four,lgb.predict(X_four))

# model_name = "LGBMClassifier"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,lgb.predict(X_four))

# print("########################LGBMClassifier########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test


params_mlp = {'classifier__hidden_layer_sizes':[50,100,150,200],
              'classifier__solver': ["sgd","adam"],
              "classifier__learning_rate_init" :[0.01, 0.015, 0.025, 0.05, 0.1],
   "classifier__early_stopping" : [True],
             "classifier__shuffle" : [True],
              "classifier__class_weight" : [{0: 1234/4774, 1: 1}]
             }

mlp_classifier= MLPClassifier()
pipeline_mlp = Pipeline([
    ('classifier', mlp_classifier)
])
model_mlp = RandomizedSearchCV(pipeline_mlp,
                               param_distributions=params_mlp,
                               n_iter=n_iter_search, verbose=1,
                               scoring=scoring, cv=5)
model_mlp.fit(X_train_copy,y_train)
mlp = model_mlp.best_estimator_
matrix = metrics.confusion_matrix(y_test,mlp.predict(X_test_copy))
precision = precision_score(y_test,mlp.predict(X_test_copy),zero_division = 1)
recall = recall_score(y_test,mlp.predict(X_test_copy))
f1 = f1_score(y_test,mlp.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,mlp.predict(X_test_copy))

model_name = "MLPClassifier"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,mlp.predict(X_test_copy))

print("########################MLPClassifier########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,mlp.predict(X_four))
# precision = precision_score(y_four,mlp.predict(X_four),zero_division = 1)
# recall = recall_score(y_four,mlp.predict(X_four))
# f1 = f1_score(y_four,mlp.predict(X_four))
# mcc = matthews_corrcoef(y_four,mlp.predict(X_four))

# model_name = "MLPClassifier"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,mlp.predict(X_four))

# print("########################MLPClassifier########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)


X_train_copy = X_train
X_test_copy = X_test


params_cat = {'classifier__loss_function':["RMSE","Logloss","CrossEntropy","MAE"],
              'classifier__iterations': [1000],
              "classifier__learning_rate" :[0.01, 0.015, 0.025, 0.05, 0.1],
             "classifier__sampling_frequency" : ["PerTree","PerTreeLevel"]
             }
cat_classifier = CatBoostClassifier()
pipeline_cat = Pipeline([
    ('classifier', cat_classifier)
])
model_cat = RandomizedSearchCV(pipeline_cat,
                               param_distributions=params_cat,
                               n_iter=n_iter_search, verbose=0,
                               scoring=scoring, cv=5)
model_cat.fit(X_train_copy,y_train)
cat = model_cat.best_estimator_
matrix = metrics.confusion_matrix(y_test,cat.predict(X_test_copy))
precision = precision_score(y_test,cat.predict(X_test_copy))
recall = recall_score(y_test,cat.predict(X_test_copy))
f1 = f1_score(y_test,cat.predict(X_test_copy))
mcc = matthews_corrcoef(y_test,cat.predict(X_test_copy))

model_name = "CatBoostClassifier"
dataset = "MorganFingerPrint"
preprocessing = "PCA"
per_index = performance(per_index,model_name,dataset,preprocessing,y_test,cat.predict(X_test_copy))

print("########################CatBoostClassifier########################")
print("Best F1 score {} , MCC : {}".format(f1,mcc))
print("Precision : {},  Recall : {}".format(precision,recall))
print(matrix)
print("\n\n")
print(per_index)
# matrix = metrics.confusion_matrix(y_four,cat.predict(X_four))
# precision = precision_score(y_four,cat.predict(X_four))
# recall = recall_score(y_four,cat.predict(X_four))
# f1 = f1_score(y_four,cat.predict(X_four))
# mcc = matthews_corrcoef(y_four,cat.predict(X_four))

# model_name = "CatBoostClassifier"
# dataset = "MorganFingerPrint (All data)"
# preprocessing = "PCA"
# per_index = performance(per_index,model_name,dataset,preprocessing,y_four,cat.predict(X_four))

# print("########################CatBoostClassifier########################")
# print("Best F1 score {} , MCC : {}".format(f1,mcc))
# print("Precision : {},  Recall : {}".format(precision,recall))
# print(matrix)
# print("\n\n")
# print(per_index)
per_index



In [ ]:
p2 = per_index
p2